# Task 5.2

##### Importing the data 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("E:\\Internship\\Week 5\\online+retail\\Online Retail.csv")
data.head()

##### Dealing with misssing Values

In [ ]:

# Handling missing data: Drop rows with missing CustomerID
data_cleaned = data.dropna(subset=['CustomerID']).copy()

# Convert 'InvoiceDate' to datetime format and handle any errors in conversion
data_cleaned['InvoiceDate'] = pd.to_datetime(data_cleaned['InvoiceDate'], errors='coerce')

# Verify the conversion worked
print(data_cleaned['InvoiceDate'].dtype)  # Should be datetime64[ns]

# Create a 'YearMonth' column for purchase dates
data_cleaned['YearMonth'] = data_cleaned['InvoiceDate'].dt.to_period('M')

# Check the cleaned data
print(data_cleaned.head())

#### Cohort Analysis

In [ ]:
from operator import attrgetter

# Define a function to extract the first purchase month for each customer
data_cleaned['CohortMonth'] = data_cleaned.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')

# Extract the YearMonth from InvoiceDate to create a 'YearMonth' column
data_cleaned['YearMonth'] = data_cleaned['InvoiceDate'].dt.to_period('M')

# Calculate the time offset between each purchase and the customer's cohort
data_cleaned['CohortIndex'] = (data_cleaned['YearMonth'] - data_cleaned['CohortMonth']).apply(attrgetter('n'))

# Build a cohort-based pivot table
cohort_data = data_cleaned.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].nunique().unstack(fill_value=0)

# Calculate cohort percentages
cohort_size = cohort_data.iloc[:, 0]
cohort_retention = cohort_data.divide(cohort_size, axis=0)

# Display the retention table
print(cohort_retention)


#### RFM Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming data_cleaned is already prepared as before

# Calculate TotalPrice
data_cleaned['TotalPrice'] = data_cleaned['Quantity'] * data_cleaned['UnitPrice']

# Reference date for Recency calculation (latest purchase date)
reference_date = data_cleaned['InvoiceDate'].max()

# Create RFM table
rfm = data_cleaned.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',   # Frequency
    'TotalPrice': 'sum'       # Monetary
}).reset_index()

# Rename the columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Define the number of quartiles (4 in this case)
quartiles = 4

# Quartile-based segmentation with dynamic binning
rfm['R_Quartile'] = pd.qcut(rfm['Recency'], quartiles, labels=range(quartiles, 0, -1), duplicates='drop')
rfm['F_Quartile'] = pd.qcut(rfm['Frequency'].rank(method="first"), quartiles, labels=range(1, quartiles + 1), duplicates='drop')
rfm['M_Quartile'] = pd.qcut(rfm['Monetary'].rank(method="first"), quartiles, labels=range(1, quartiles + 1), duplicates='drop')

# Combine RFM segments into a single score
rfm['RFM_Score'] = rfm[['R_Quartile', 'F_Quartile', 'M_Quartile']].sum(axis=1)

# Visualize RFM segments
plt.figure(figsize=(8, 6))
plt.hist(rfm['RFM_Score'], bins=range(3, 13), edgecolor='black')
plt.title('Distribution of RFM Scores')
plt.xlabel('RFM Score')
plt.ylabel('Number of Customers')
plt.show()

# Check high-value customers
high_value_customers = rfm[rfm['RFM_Score'] >= 9]
print(high_value_customers.head())
